# Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/')
%cd /content/drive/My\ Drive/

Mounted at /content/drive
/content/drive/My Drive


# Load Data

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torch.optim as optim
import numpy as np
import sys
from tqdm import tqdm

In [ ]:
from collections import defaultdict

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

batch_size = 128
default_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download = True, transform = default_transform)

split_ratio = 0.8
total_size = len(trainset)
train_size = int(split_ratio * total_size)
valid_size = total_size - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(trainset, [train_size, valid_size])

second_split = 0.2
total_size = len(train_dataset)
train_size = int(second_split * total_size)
discard_size = total_size - train_size
print(train_size)
train_dataset, discard_dataset = torch.utils.data.random_split(train_dataset, [train_size, discard_size])

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)
validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=8)


testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform = default_transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=8)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

class_distribution = defaultdict(int)
for _, target in trainloader.dataset:
    class_name = classes[target]
    class_distribution[class_name] += 1
for class_name, count in class_distribution.items():
    print(f"{class_name}: {count} samples")

Files already downloaded and verified
8000


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Files already downloaded and verified
plane: 802 samples
deer: 805 samples
horse: 797 samples
car: 832 samples
ship: 815 samples
truck: 794 samples
frog: 802 samples
cat: 768 samples
bird: 770 samples
dog: 815 samples


# ResNet50

In [ ]:
from torch.optim.lr_scheduler import StepLR

resnet50 = models.resnet50(pretrained=True)
num_classes = 10
resnet50.fc = nn.Linear(resnet50.fc.in_features, num_classes)
resnet50 = resnet50.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet50.parameters(), lr=1e-2, momentum=0.9)

scheduler = StepLR(optimizer, step_size=20, gamma=0.1)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
num_epochs = 50

for epoch in range(num_epochs):
  resnet50.train()
  running_loss = 0.0

  for inputs, labels in trainloader:
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()

    outputs = resnet50(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  scheduler.step()

  print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(trainloader)}')

resnet50.eval()
correct = 0
total = 0

with torch.no_grad():
  for inputs, labels in testloader:
    inputs, labels = inputs.to(device), labels.to(device)

    outputs = resnet50(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {100 * accuracy:.2f}%')

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 1/50, Loss: 1.388039715706356
Epoch 2/50, Loss: 0.6567548343113491
Epoch 3/50, Loss: 0.32248325409397244
Epoch 4/50, Loss: 0.18482199464998547
Epoch 5/50, Loss: 0.1424010679244049
Epoch 6/50, Loss: 0.13183568808294477
Epoch 7/50, Loss: 0.11499637152467455
Epoch 8/50, Loss: 0.08581125916587928
Epoch 9/50, Loss: 0.05540329519481886
Epoch 10/50, Loss: 0.027293926537302986
Epoch 11/50, Loss: 0.04555214010179043
Epoch 12/50, Loss: 0.03811050847815793
Epoch 13/50, Loss: 0.03705774787222109
Epoch 14/50, Loss: 0.04424786042895109
Epoch 15/50, Loss: 0.036572228666276685
Epoch 16/50, Loss: 0.02425341016709036
Epoch 17/50, Loss: 0.02403383790182748
Epoch 18/50, Loss: 0.026963027491648164
Epoch 19/50, Loss: 0.027426055591878672
Epoch 20/50, Loss: 0.014769691377989062
Epoch 21/50, Loss: 0.00847548813169216
Epoch 22/50, Loss: 0.004115623122151379
Epoch 23/50, Loss: 0.002345015197375139
Epoch 24/50, Loss: 0.002030919585469371
Epoch 25/50, Loss: 0.0021721011920211217
Epoch 26/50, Loss: 0.0012505

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import models

learning_rates = [1e-2, 5e-2, 1e-1]
num_epochs_list = [30, 50, 70]

best_accuracy = 0.0
best_params = {}
best_model = None

for lr in learning_rates:
    for num_epochs in num_epochs_list:

        resnet50 = models.resnet50(pretrained=True)
        num_classes = 10
        resnet50.fc = nn.Linear(resnet50.fc.in_features, num_classes)
        resnet50 = resnet50.to(device)


        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(resnet50.parameters(), lr=lr, momentum=0.9)
        scheduler = StepLR(optimizer, step_size=20, gamma=0.1)


        for epoch in range(num_epochs):
            resnet50.train()
            running_loss = 0.0

            for inputs, labels in trainloader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = resnet50(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

            scheduler.step()


        resnet50.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in validloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = resnet50(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = {'lr': lr, 'num_epochs': num_epochs}
            best_model = resnet50

print(f'Best Hyperparameters: {best_params}')
print(f'Best Valid Accuracy: {100 * best_accuracy:.2f}%')

correct = 0
total = 0
with torch.no_grad():
  for inputs, labels in testloader:
    inputs, labels = inputs.to(device), labels.to(device)

    outputs = best_model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {100 * accuracy:.2f}%')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the wo

Best Hyperparameters: {'lr': 0.01, 'num_epochs': 50}
Best Valid Accuracy: 78.02%
Test Accuracy: 76.82%


# Save Model

In [ ]:
torch.save(resnet50.state_dict(), 'resnet.pth')

# Load Model

In [ ]:
state_dict = torch.load('resnet.pth', map_location=torch.device(device))

resnet50 = models.resnet50(pretrained=True)
resnet50.fc = nn.Linear(resnet50.fc.in_features, num_classes)
resnet50.to(device)

resnet50.load_state_dict(state_dict)

<All keys matched successfully>